In [ ]:
from vpython import *
from numpy import arange
scene = canvas(title='Plot3D')
scene.width = scene.height = 600
scene.forward = vec(-0.7,-0.5,-1)
L = 50
dL = 1 # vertex objects are on an L by L grid
scene.center = vec(0.05*L,0.2*L,0)
scene.range = 1.3*L
## The next line contains LaTeX math notation. See http://www.glowscript.org/docs/VPythonDocs/MathJax.html
#scene.caption = """\\( f(x,y,t) = 0.7+0.2\\sin{(10x)}\\cos{(10y)}\\cos{(2t)} \\)
scene.caption = """<i>f</i>(<i>x,y,t</i>) = 0.7+0.2sin{(10<i>x</i>)cos{(10<i>y</i>)cos{(2<i>t</i>)
<b>Click to toggle between pausing or running.</b>
In GlowScript programs:
   Right button drag or Ctrl-drag to rotate "camera" to view scene.
   To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
     On a two-button mouse, middle is left + right.
   Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""
   
#MathJax.Hub.Queue(["Typeset",MathJax.Hub]) # format the LaTeX; see http://www.glowscript.org/docs/VPythonDocs/MathJax.html

class plot3D:
    def __init__(self, f, xmin, xmax, ymin, ymax, zmin, zmax):
        # The x axis is labeled y, the z axis is labeled x, and the y axis is labeled z.
        # This is done to mimic fairly standard practive for plotting
        #     the z value of a function of x and y.
        self.f = f
        if not xmin: self.xmin = 0
        else: self.xmin = xmin
        if not xmax: self.xmax = 1
        else: self.xmax = xmax
        if not ymin: self.ymin = 0
        else: self.ymin = ymin
        if not ymax: self.ymax = 1
        else: self.ymax = ymax
        if not zmin: self.zmin = 0
        else: self.zmin = zmin
        if not zmax: self.zmax = 1
        else: self.zmax = zmax
        
        R = L/100
        xaxis = cylinder(pos=vec(0,0,0), axis=vec(0,0,L), radius=R, color=color.yellow)
        yaxis = cylinder(pos=vec(0,0,0), axis=vec(L,0,0), radius=R, color=color.yellow)
        zaxis = cylinder(pos=vec(0,0,0), axis=vec(0,L,0), radius=R, color=color.yellow)
        k = 1.02
        #h = 0.05*L
        #text(pos=xaxis.pos+k*xaxis.axis, text='x', height=h, align='center', billboard=True)
        #text(pos=yaxis.pos+k*yaxis.axis, text='y', height=h, align='center', billboard=True)
        #text(pos=zaxis.pos+k*zaxis.axis, text='z', height=h, align='center', billboard=True)
        h = 20
        label(pos=xaxis.pos+k*xaxis.axis, text='x', height=h, align='center', box=False)
        label(pos=yaxis.pos+k*yaxis.axis, text='y', height=h, align='center', box=False)
        label(pos=zaxis.pos+k*zaxis.axis, text='z', height=h, align='center', box=False)
    
        # Create vertex objects on an L by L grid, plus vertex objects at the top and the right of the grid
        # These extra vertex objects are not used in making quads but they are used in the calculation of
        #    normals, without having to do special edge tests when calculating normals.
        self.vertices = [] 
        for x in arange(0, L+dL+dL/2, dL):
            for y in arange(0, L+dL+dL/2, dL):
                val = self.evaluate(x,y)
                self.vertices.append(self.make_vertex( x, y, val ))
        
        self.make_quads()
        self.make_normals()
        
    def evaluate(self, x, y):
        return (L/(self.zmax-self.zmin)) * (self.f(self.xmin+x*(self.xmax-self.xmin)/L, self.ymin+y*(self.ymax-self.ymin)/L)-self.zmin)
    
    def make_quads(self):
        # Create the quad objects, based on the vertex objects already created.
        for x in arange(0, L-dL/2, dL):
            for y in arange(0, L-dL/2, dL):
                v0 = self.get_vertex(x,y)
                v1 = self.get_vertex(x+dL,y)
                v2 = self.get_vertex(x+dL, y+dL)
                v3 = self.get_vertex(x, y+dL)
                quad(vs=[v0, v1, v2, v3])
        
    def make_normals(self):
        # Set the normals for each vertex to be the average of neighboring normals to the surface.
        # The vectors a, b, c, and d point left, down, right, and up around a vertex in the xy plance.
        # Normals are perpendicual to each quad object.
        for x in arange(0, L+dL/2, dL):
            for y in arange(0, L+dL/2, dL):
                v = self.get_pos(x,y)
                a = self.get_pos(x+dL,y) - v
                b = self.get_pos(x,y+dL) - v
                #self.get_vertex(x,y).normal = cross(a,b)
    
    def replot(self):
        for x in arange(0, L+dL+dL/2, dL):
            for y in arange(0, L+dL+dL/2, dL):
                val = self.evaluate(x,y)
                self.get_vertex(x,y).pos.y = val
        #self.make_normals()
                
    def make_vertex(self,x,y,value):
        return vertex(pos=vec(y,value,x), color=color.cyan, normal=vec(0,1,0))
        
    def get_vertex(self,x,y):
        return self.vertices[int((x*(L+2*dL)/dL + y)/dL)]
        
    def get_pos(self,x,y):
        return self.get_vertex(x,y).pos

t = 0
dt = 0.0333
def f(x, y):
    # Return the value of the function of x and y:
    return 0.7+0.2*cos(5*t)*sin(10*x)*cos(10*y)

p = plot3D(f, 0, 1, 0, 1, 0, 1) # function, xmin, xmax, ymin, ymax (defaults 0, 1, 0, 1, 0, 1)

run = True
def running(ev):
    global run
    run = not run

scene.bind('mousedown', running)

while True:
    rate(30)
    if run:
        p.replot()
        t += dt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>